# Generate point-in-time best execution data using dedicated service - F#

### Overview
Best execution aims to measure the performance of executed trades, an execution algorithm or an execution venue.

This sample is based on an `point-in-time` approach and designed to highlight individual trades within market activity over a look back period. 

<div class="alert alert-block alert-info">
    <b>Note:</b> To explore the interval approach, suitable to analyze market activity over a look back period. Please refer to <i>bestex-interval</i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** best execution *point-in-time* results.

### Inputs/outputs
Best execution PIT sample requires instrument's identifier, **single trade** and time window as per inputs. Results are as follows:
* a snapshot of occured trades around the given trade based on the input time window

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Packages required
1. Systemathics:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source:
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package
    
***


# Run BestEx Point-in-time sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: TaskBuilder.fs"

In [ ]:
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.Tick.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Net.Client
open Grpc.Core
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
let token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN")
let api = System.Environment.GetEnvironmentVariable("GRPC_APIS")

// Add token to the request header
let headers = new Metadata()
headers.Add("Authorization", $"Bearer (token)")

// Define API entry
let channel = GrpcChannel.ForAddress($"https://(api)")

In [ ]:
// Display authentication token 
token |> display

### Step 3: Create and process request

#### 3.1 Instrument selection

In [ ]:
// Set the instrument  identifier: a ticker and an exchange code
let ticker = "AAPL"
let exchange = "BATS"

// Set the time window to delimit the input trade
let window = TimeSpan.FromMinutes(15.)

#### 3.2 Trade parameters

We assume our trade is the following:

In [ ]:
let timestamp = Convert.ToDateTime("2021-03-05T12:13:21.877300Z")
let price = 119.81
let size = 100

#### 3.3 Time period delimitation

In [ ]:
// Build the tick trades request date interval (we are using Google date time format)
let dateIntervals = 
    new DateInterval(
        StartDate = new Date ( Year = timestamp.Year, Month = timestamp.Month, Day = timestamp.Day ),
        EndDate = new Date ( Year = timestamp.Year, Month = timestamp.Month, Day = timestamp.Day ) 
    )

// Get the first and the last times from the input trade
let firstTime = timestamp.TimeOfDay.Add(-window)
let lastTime = timestamp.TimeOfDay.Add(window)

// Build the tick trades request time interval (we are using Google date time format)
// UTC time zone
let timeInterval = 
    new TimeInterval(
        StartTime = new TimeOfDay ( Hours = firstTime.Hours, Minutes = firstTime.Minutes, Seconds = firstTime.Seconds ), 
        EndTime = new TimeOfDay ( Hours = lastTime.Hours, Minutes = lastTime.Minutes, Seconds = lastTime.Seconds )
    )

#### 3.4 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
// Generate constraints based on the previous time selection
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)
constraints.TimeIntervals.Add(timeInterval)

In [ ]:
// Generate the tick trades request
let request = new TickTradesRequest ( Constraints = constraints )
let identifier = new Identifier ( Exchange = exchange, Ticker = ticker)
request.Identifiers.Add( [| identifier |] ) 

In [ ]:
// Instantiate the tick trades service
let service = new TickTradesService.TickTradesServiceClient(channel)

### Step 4: Visualize data

#### 4.1 Retrive trades

In [ ]:
open FSharp.Control.Tasks.V2
open System.Collections.Generic

let iterAsync (t: IAsyncEnumerable<'T>) (f : 'T -> unit) =
    let task = 
        task {
            let e = t.GetAsyncEnumerator()
            let mutable go = true
            try
                let! step = e.MoveNextAsync()
                go <- step
                while go do 
                    f e.Current
                    let! step = e.MoveNextAsync()
                    go <- step
            finally
                e.DisposeAsync().AsTask().Wait()
        }
    task.Wait()

let add (collection : List<Trade>) (item : TickTradesResponse) =
    if( item |> isNull |> not && item.Trade |> isNull |> not ) then
        collection.Add item.Trade


In [ ]:
// Get the trades
let trades = new List<Trade>()

let call = service.TickTrades(request, headers)
let e = call.ResponseStream.ReadAllAsync()

iterAsync e (add trades)


The following code snippet displays request results:

In [ ]:
trades |> display

#### 4.2 Plot individual trade within market snapshot
Display trade prices and highlight the individual trade over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.
Plot a simple line graph with the trade prices and mark out the individual trade as follows: 

In [ ]:
// Create Trade class to handle the input trades
public class Input
(
    public DateTime Timestamp ( get set )
    public double Price ( get set )
    public long Size ( get set )
)

// Input trade : trade to validate
let input = new Input() ( Timestamp = timestamp, Price = price, Size = size )
let inputs = new List<Input>()(input)

In [ ]:
let bestexPit = 
    [
        new Scattergl (name = "Price", x = (trades |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())), y = (trades |> Seq.map (fun elt -> elt.Price)), mode = "lines" )
        new Scattergl (name = "Input trade price", x = [ timestamp ], y = [ price ], mode = "markers" )
        new Scattergl (name = "Input trade size", x = [ timestamp ], y = [size ], mode = "markers", yaxis = "y2" )
        new Scattergl (name = "Size", mode = "lines", x = (trades |> Seq.map (fun elt -> elt.TimeStamp.ToDateTime())), y = (trades |> Seq.map (fun elt -> elt.Size)), yaxis = "y2")
    ]

let layout = 
    new Layout.Layout( 
        title = $"Input trade within market snapshot for (ticker)-(exchange)",
        xaxis = new Xaxis (title = "Time" ),
        yaxis = new Yaxis ( title = "Price", autorange = true, side = "left"),
        yaxis2 = new Yaxis ( title = "Size", autorange = true, side = "right")
    )

Chart.Plot(bestexPit, layout)
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display